<a href="https://colab.research.google.com/github/Emdey/AES-encryption-SYSTEM/blob/main/AES_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cell 1:** Create Streamlit Configuration
 Directory (Fix)
This cell fixes the FileNotFoundError by creating the .streamlit folder, which is necessary before writing the configuration file inside it.

**Configure App Color Theme**

This cell writes the configuration file that tells Streamlit to use your customized dark blue color scheme (#000080 for the primary accent and #ADD8E6 for the sidebar background).

In [ ]:
import os

# Create the .streamlit directory if it doesn't exist
os.makedirs(".streamlit", exist_ok=True)

# Define the content to write as a multi-line string
config_content = """[theme]
# Teal primary color for buttons, links, and accents
primaryColor="#000080"
# White background
backgroundColor="#FFFFFF"
# Light gray for sidebar and input area backgrounds
secondaryBackgroundColor="#ADD8E6"
# Dark text color
textColor="#000000"
font="sans serif"
"""

# Write the content to the file
with open(".streamlit/config.toml", "w") as f:
    f.write(config_content)

print("Writing .streamlit/config.toml") # Mimic %%writefile output

**Cell 3:** Create the Streamlit Application Script (aes_app.py)
This cell creates the main Python script containing the AES encryption logic and the Streamlit UI, including your customized title and page icon.

In [ ]:
%%writefile aes_app.py
import streamlit as st
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os
import binascii
import time

# --- 1. CORE AES FUNCTIONS ---
def generate_aes_key_iv(key_size_bytes=32):
    key = os.urandom(key_size_bytes)
    iv = os.urandom(16)
    return key, iv

def aes_encrypt(plaintext_bytes, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext_bytes) + padder.finalize()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext

def aes_decrypt(ciphertext, key, iv):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_padded_data = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext_bytes = unpadder.update(decrypted_padded_data) + unpadder.finalize()
    return plaintext_bytes

# --- 2. STREAMLIT WEB INTERFACE (Customized) ---
# --- TITLE AND ICON CHANGES HERE ---
st.set_page_config(page_title="SecretShare: Secure Encryption Tool", page_icon="🔑", layout="wide")
st.title("🔑 SecretShare: Ultimate AES Messenger")
st.markdown("---")

st.sidebar.image("https://img.icons8.com/?size=100&id=10489&format=png&color=000000", use_container_width=True)
st.sidebar.markdown("---")

# **Key Generation Function (Callback)**
def generate_new_keys_callback():
    # Use the value already set by the st.selectbox
    key_size_bits = st.session_state.key_size_bits
    key_size_bytes = key_size_bits // 8

    with st.spinner(f'Generating {key_size_bits}-bit AES Key and IV...'):
        st.session_state.secret_key, st.session_state.iv = generate_aes_key_iv(key_size_bytes)
        st.session_state.ciphertext_hex = None # Clear output on new key gen
        time.sleep(0.5)
    st.balloons()


# **Key Generation Section**
key_size_bits = st.sidebar.selectbox(
    "Select AES Key Size:",
    options=[128, 256],
    index=1,
    key="key_size_bits" # This sets st.session_state.key_size_bits
)


# The st.selectbox already guarantees st.session_state.key_size_bits exists now.
if 'secret_key' not in st.session_state:
    generate_new_keys_callback() # Generate keys on first run

st.sidebar.button(
    f"🔒 **Generate New {key_size_bits}-bit Key**", # Updated button text
    type="primary",
    on_click=generate_new_keys_callback
)


if 'secret_key' in st.session_state:
    key = st.session_state.secret_key
    iv = st.session_state.iv

    st.sidebar.subheader("Current Symmetric Keys")
    st.sidebar.markdown(f"**Key Size:** {key_size_bits}-bit") # Use local key_size_bits variable

    st.sidebar.text_area(
        "Secret Key (Hex) - KEEP PRIVATE:",
        value=binascii.hexlify(key).decode(),
        height=70,
        key="secret_key_sidebar"
    )
    st.sidebar.text_area(
        "IV (Hex) - Share alongside Ciphertext:",
        value=binascii.hexlify(iv).decode(),
        height=70,
        key="iv_sidebar"
    )

    # **Main Application Area: Local Encrypt/Decrypt**
    st.header("1. ✉️ Local Encryption Operations")

    col1, col2 = st.columns([2, 1])

    with col1:
        message_input = st.text_area(
            "Enter your secret message here:", # Updated text
            value="This is the secret message for the AES demonstration!",
            height=150
        )
        plaintext_bytes = message_input.encode('utf-8')

    with col2:
        # --- DROPDOWN MENU ---
        st.markdown("<div style='margin-top: 25px;'></div>", unsafe_allow_html=True)
        operation_mode = st.selectbox(
            "Select Operation Mode:",
            options=["🔐 Encrypt Only", "🔄 Encrypt and Decrypt Cycle"],
            index=0
        )
        # --- Single Button to execute the selected mode ---
        run_op_btn = st.button("🚀 **EXECUTE OPERATION**", use_container_width=True, type="primary") # Updated button text


    st.markdown("---")
    st.subheader("Output")

    # EXECUTION LOGIC
    if run_op_btn:
        # --- ENCRYPT ALWAYS RUNS ---
        with st.spinner('Encrypting...'):
            ciphertext_bytes = aes_encrypt(plaintext_bytes, key, iv)
            ciphertext_hex = binascii.hexlify(ciphertext_bytes).decode()
            st.session_state.ciphertext_hex = ciphertext_hex # Store for display
            time.sleep(1)

        st.success(f"**Ciphertext (Hex):**")
        st.code(ciphertext_hex)

        # --- DECRYPT RUNS ONLY IF THE CYCLE IS SELECTED ---
        if operation_mode == "🔄 Encrypt and Decrypt Cycle":
            st.markdown("<div style='margin-top: 20px;'></div>", unsafe_allow_html=True)
            with st.spinner('Decrypting...'):
                decrypted_bytes = aes_decrypt(ciphertext_bytes, key, iv)
                decrypted_message = decrypted_bytes.decode('utf-8')
                time.sleep(1)

            st.info(f"**Decrypted Message:**")
            st.code(decrypted_message)

            # Simple check to show success
            if decrypted_message.strip() == message_input.strip():
                st.success("✅ Decryption Successful: Original message recovered!")
            else:
                st.error("❌ Decryption failed or message content mismatch.")
        else:
            st.info("Mode set to **Encrypt Only**. Decryption was skipped.")


else:
    st.warning("Please click '🔒 Generate New Key' in the sidebar to begin.")


# --- SECTION: EXTERNAL DECRYPTION (MANUAL INPUT - NO CHANGE) ---
st.markdown("---")
st.header("2. 📤 Decrypt External Message")
st.caption("Use this to decrypt ciphertext provided you have the Secret Key and IV used for encryption.")

with st.form("external_decrypt_form"):\
    # Input for Ciphertext
    cipher_input = st.text_area(
        "Enter Ciphertext (Hex string):",
        value=st.session_state.ciphertext_hex if st.session_state.get('ciphertext_hex') else "", # Use .get() for safety
        height=100
    )

    # Input for Secret Key and IV
    key_input = st.text_input(
        "Enter Secret Key (Hex string):",
        placeholder="Copy this value from the sidebar."
    )
    iv_input = st.text_input(
        "Enter IV (Hex string):",
        placeholder="Copy this value from the sidebar."
    )

    decrypt_button = st.form_submit_button("🔓 **Decrypt External Message**", type="primary")

if decrypt_button:
    if not cipher_input or not key_input or not iv_input:
        st.error("Please enter a valid Ciphertext, Secret Key, and IV.")
    else:
        try:
            external_ciphertext = binascii.unhexlify(cipher_input)
            external_key = binascii.unhexlify(key_input)
            external_iv = binascii.unhexlify(iv_input)

            with st.spinner('Decrypting External Message...'):
                decrypted_bytes_external = aes_decrypt(external_ciphertext, external_key, external_iv)
                decrypted_message_external = decrypted_bytes_external.decode('utf-8')
                time.sleep(2)

            st.success("✅ Decryption Complete!")
            st.info(f"**Decrypted Message:**")
            st.code(decrypted_message_external)

        except binascii.Error:
            st.error("Input Error: Key, IV, or Ciphertext is not a valid hex string.")
        except Exception as e:
            st.error(f"A decryption error occurred. Check that the Secret Key and IV are correct and match the key size used for encryption. Details: {e}")

**Cell 4:** Install Required Libraries
This cell installs the Python packages needed to run the application and handle the deployment.

In [ ]:
!pip install -q streamlit pyngrok cryptography

**Cell 5:** Aggressive Cleanup (Pre-Deployment)
This is the essential cleanup step that prevents conflicts if you are re-running the notebook. It should be run every time before deployment.

In [ ]:
import os
from pyngrok import ngrok
import time

# 1. Kill the Streamlit server process
print("Cleaning up old Streamlit processes...")
os.system("pkill streamlit")
time.sleep(2) # Give the process time to shut down

# 2. Kill all existing ngrok tunnels
print("Killing all existing ngrok tunnels...")
ngrok.kill()

print("Cleanup complete. You can now re-run the deployment cell.")

In [ ]:
from pyngrok import ngrok

# Disconnects all current ngrok tunnels.
ngrok.kill()

print("All active ngrok tunnels have been closed. You can now try re-running the deployment cell.")

**Cell 6:** Deploy the Application
This is the final cell that launches the Streamlit server in the background and creates the public ngrok URL for access. Note: You must replace the placeholder NGROK_AUTH_TOKEN with your actual ngrok token.

In [ ]:
import os
from pyngrok import ngrok
import time
from datetime import datetime

# --- STEP 1: SET NGROK TOKEN (Using the token from your previous code) ---
NGROK_AUTH_TOKEN = "35UvQwxeSmLoD8iE2X0O9Ame1rI_7DBXPJyJUAsLjSZMXGbxt"
try:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
except Exception as e:
    print(f"Error setting ngrok token: {e}")
    raise

# --- STEP 2: AGGRESSIVE CLEANUP AND LAUNCH ---

# 1. Kill the Streamlit server process FIRST
print("\nCleaning up old Streamlit processes...")
os.system("pkill streamlit")
time.sleep(2)

# 2. KILL ALL EXISTING PYNGROK TUNNELS (Critical step)
print("Killing all existing ngrok tunnels...")
ngrok.kill()
time.sleep(5) # Wait a bit longer for the ngrok process to die

# 3. Start Streamlit in the background (&)
# This will now pick up the changes in aes_app.py and config.toml
print("Starting Streamlit server on port 8501...")
os.system("streamlit run aes_app.py &")

# 4. Wait for the server to initialize
time.sleep(15)

# 5. Create the public ngrok tunnel, using a unique subdomain prefix
# We create a unique name using the current timestamp to avoid old reservations.
unique_subdomain = f"aes-deploy-{datetime.now().strftime('%H%M%S')}"

print(f"Creating public ngrok tunnel with unique identifier...")
try:

    # If the free tier ignores it, it still resets the connection attempt.
    public_url = ngrok.connect(addr="8501", proto="http")

    print("\n" + "="*80)
    print("🔒 Your Custom AES App is Live and Deployed!")
    print(f"🔗 Public URL: {public_url}")
    print("="*80 + "\n")

except Exception as e:
    # If this still fails, the last resort is restarting the entire Colab runtime.
    print(f"Failed to create ngrok tunnel: {e}")
    print("\n---")
    print("⚠️ FINAL STEP IF ERROR PERSISTS: Please try restarting your Colab runtime (Runtime -> Restart runtime) and running the cells from the beginning.")